In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

options = Options()
options.headless = True  
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

url = "https://www.camclarkfordrichmond.com/used/"
driver.get(url)

time.sleep(5) 

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

car_listings = driver.find_elements(By.CLASS_NAME, 'vehicle-list-cell')

cars = []
for item in car_listings:
    try:
        name_tag = item.find_element(By.CLASS_NAME, 'vehicle-year-make-model')
        year = name_tag.find_element(By.XPATH, ".//span[@itemprop='releaseDate']").text.strip()
        manufacturer = name_tag.find_element(By.XPATH, ".//span[@itemprop='manufacturer']").text.strip()
        model = name_tag.find_element(By.XPATH, ".//span[@itemprop='model']").text.strip()
        name = f"{year} {manufacturer} {model}"
    except:
        name = "N/A"
    
    try:
        price_tag = item.find_element(By.CSS_SELECTOR, 'span[itemprop="price"]')
        price = price_tag.text.strip()
    except:
        price = "N/A"
    
    cars.append({'name': name, 'price': price})

driver.quit()

if cars:
    df = pd.DataFrame(cars)
    print(df)
else:
    print("No car data extracted.")

                                                 name     price
0         2015 Mazda CX-5 GT (A6) 4dr All-Wheel Drive   $17,995
1   2017 Mazda CX-3 GT 4dr All-Wheel Drive Sport U...   $25,995
2                         2019 Ford Escape SE 4dr 4x4   $23,995
3           2018 Nissan NV200 S 4dr Compact Cargo Van   $25,995
4   2022 Hyundai Elantra Preferred w/Sun & Tech Pk...   $26,995
5                   2021 Toyota Corolla Hatchback 5dr   $26,995
6   2022 Hyundai Kona 2.0L Preferred Sun & Leather...   $28,995
7          2017 Jeep Grand Cherokee Trailhawk 4dr 4x4   $28,995
8            2022 Ford Escape SEL 4dr All-Wheel Drive   $32,995
9      2021 Ford Escape SE Hybrid 4dr All-Wheel Drive   $27,995
10                                  2019 Ford Mustang   $36,995
11  2020 Land Rover Range Rover Evoque SE All-Whee...   $36,995
12  2021 Mazda CX-5 Signature 4dr i-ACTIV All-Whee...   $36,995
13                  2017 Ford Mustang GT 2dr Fastback   $40,995
14  2019 Ford F-150 Lariat 4x4 SuperCrew

In [22]:
df = pd.DataFrame(cars)
print(df)

                                                 name     price
0         2015 Mazda CX-5 GT (A6) 4dr All-Wheel Drive   $17,995
1   2017 Mazda CX-3 GT 4dr All-Wheel Drive Sport U...   $25,995
2                         2019 Ford Escape SE 4dr 4x4   $23,995
3           2018 Nissan NV200 S 4dr Compact Cargo Van   $25,995
4   2022 Hyundai Elantra Preferred w/Sun & Tech Pk...   $26,995
5                   2021 Toyota Corolla Hatchback 5dr   $26,995
6   2022 Hyundai Kona 2.0L Preferred Sun & Leather...   $28,995
7          2017 Jeep Grand Cherokee Trailhawk 4dr 4x4   $28,995
8            2022 Ford Escape SEL 4dr All-Wheel Drive   $32,995
9      2021 Ford Escape SE Hybrid 4dr All-Wheel Drive   $27,995
10                                  2019 Ford Mustang   $36,995
11  2020 Land Rover Range Rover Evoque SE All-Whee...   $36,995
12  2021 Mazda CX-5 Signature 4dr i-ACTIV All-Whee...   $36,995
13                  2017 Ford Mustang GT 2dr Fastback   $40,995
14  2019 Ford F-150 Lariat 4x4 SuperCrew